In [3]:
from pydantic_ai import Agent
from pydantic_graph import BaseNode, GraphRunContext, End, Graph
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.format_as_xml import format_as_xml
from dotenv import load_dotenv
import os
from pydantic import Field
from typing import  List, Dict, Optional, Union, Tuple
from enum import Enum
from pptx.util import Inches
from pptx.enum.text import PP_ALIGN, MSO_AUTO_SIZE
from pptx.dml.color import RGBColor
from pydantic import BaseModel
from dataclasses import dataclass
from IPython.display import Image, display  
import time
import requests
load_dotenv()

google_api_key=os.getenv('google_api_key')
pse=os.getenv('pse')
llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

In [155]:


@dataclass
class State:
    instruction: str
    presentation_style: str
    research: str
    presentation_plan: List[Dict]
    presentation: Dict
    presentation_config: Dict



class ChartType(str, Enum):
    COLUMN_CLUSTERED = "column_clustered"
    LINE = "line"
    PIE = "pie"
    BAR = "bar"
    SCATTER = "scatter"

class Position(BaseModel):
    left: float = Field(description='the left position of the text box in inches')
    top: float = Field(description='the top position of the text box in inches')
    width: float = Field(description='the width of the text box in inches')
    height: float = Field(description='the height of the text box in inches')


class Font_color(BaseModel):
    red: int = 0
    green: int = 0
    blue: int = 0


class TextFormat(BaseModel):
# Prevent additional properties
        
    font_size: Optional[float] = 11.0
    font_color: Optional[Font_color] = Field(default_factory= Font_color)
    is_bold: Optional[bool] = False
    is_italic: Optional[bool] = False
    alignment: Optional[int] = Field(default_factory=lambda: PP_ALIGN.LEFT.value)  # Use string instead of PP_ALIGN

class BulletPoint(BaseModel):    
    text: str
    level: int = Field(default_factory= 0)
    format: Optional[TextFormat] = Field(default_factory=TextFormat)


class ChartData(BaseModel):
    type: ChartType
    title: str
    categories: List[str]
    series: List[Dict[str, List[float]]]
    position: Position = Field(
        default_factory=lambda: Position(left=2, top=2, width=6, height=4.5,
        description="(left, top, width, height) in inches")
    )

class TextBox(BaseModel):

    text: str
    position: Position = Field(
        default_factory=lambda: Position(left=1, top=1, width=8, height=5),
        description="(left, top, width, height) in inches"
    )
    format: Optional[TextFormat] = Field(default_factory=TextFormat)

class SlideContent(BaseModel):

        
    title: Optional[str] = None
    bullet_points: Optional[List[BulletPoint]] = None
    text_boxes: Optional[List[TextBox]] = None
    # chart: Optional[ChartData] = None
    image_url: Optional[str] = None
    image_position: Optional[Position] = Field(
        default_factory=lambda: None
    )


class PresentationConfig(BaseModel):

    title: str
    slides: List[SlideContent]=None
    subtitle: Optional[str] = None
    output_path: Optional[str] = 'sample.pptx'
    theme: Optional[str] = None  # Path to theme file
    slide_width: float = 10.0
    slide_height: float = 7.5




def google_image_search(query:str):
  """Search for images using Google Custom Search API
  args: query
  return: image url
  """
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": google_api_key,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return image_url



  
text_extraction_agent=Agent(llm, result_type=list[TextBox], system_prompt='extract the text from the reaserch based on the instructions')

class step_execution_node(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])-> End:
        ctx.state.presentation_config.slides=[]
        for page in ctx.state.presentation.keys():
            slide=SlideContent()
            time.sleep(2)
            if ctx.state.presentation[page].get('image_title'):
                slide.image_url=google_image_search(ctx.state.presentation[page]['image_title'])
            if ctx.state.presentation[page].get('text_to_extract'):
                result=await text_extraction_agent.run(f'extract the text from the research: {ctx.state.research} based on the instructions: {ctx.state.presentation[page]['text_to_extract']}')
                slide.text_boxes=result.data
            ctx.state.presentation_config.slides.append(slide)
        return End(ctx.state.presentation_config)
    

class clean_up_node(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])-> step_execution_node:
        page_number=1
        for task in ctx.state.presentation_plan.tasks:
            ctx.state.presentation[f'page_{page_number}']={'title':task.text_data.Text_title, 'text_to_extract':task.text_data.Text_to_extract,'image_title':task.image_data.image_title if task.image_data else None,'image_url':task.image_data.image_url if task.image_data else None}
            page_number+=1
        return step_execution_node()
    
pptx_agent=Agent(llm, result_type=PresentationConfig, system_prompt='generate a base presentation config based on the presentation plan')
class pptx_presentation(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])-> clean_up_node:
        result=await pptx_agent.run(f'generate a base presentation config based on the presentation plan: {ctx.state.presentation_plan}')
        ctx.state.presentation_config=result.data
        return clean_up_node()
    



@dataclass
class image:
    image_title: Optional[str] = Field(description='the image title')
    image_url: Optional[str] = None

@dataclass
class text:
    Text_to_extract: str = Field(description=' a description of the text to extract from the research')
    Text_title: str = Field(description='the text title')
    Text_content: str = None

@dataclass
class steps:
    image_data: Optional[image] = Field(description='the image data, optional')
    text_data: text = Field(description='the text data')

@dataclass
class Presentation_plan:
    tasks: List[steps] = Field(description='the steps to complete')


presentation_plan_agent=Agent(llm, result_type=Presentation_plan, system_prompt='generate a presentation plan based on the research and instruction (if any), choose wether to use images or not for each step')
        
class Presentation_plan_node(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])->pptx_presentation:
        prompt=f'generate a presentation plan based on the research: {ctx.state.research} and instruction: {ctx.state.instruction} (if any), choose wether to use images or not for each step, make sure that the sources URLs are included in the presentation'
        result=await presentation_plan_agent.run(prompt)
        ctx.state.presentation_plan=result.data
        return pptx_presentation()

In [133]:
class Presentation_gen:
    def __init__(self):
        self.graph=Graph(nodes=[Presentation_plan_node, step_execution_node, clean_up_node, pptx_presentation])
        self.state=State(research='', presentation_plan=[], presentation={}, presentation_config={}, presentation_style='', instruction='')

    async def chat(self,research:str, presentation_style:Optional[str]='None', instruction:Optional[str]='None'):
        """Chat with the presentation generator,
        Args:
            research (str): The research to generate a presentation for
            presentation_style (str): The style of the presentation
            instruction (str): The instruction for the presentation
        Returns:
            str: The response from the presentation generator
        """
        self.state.research=research
        self.state.presentation_style=presentation_style
        self.state.instruction=instruction
        response=await self.graph.run(Presentation_plan_node(),state=self.state)
        return response.output


    def display_graph(self):
        """Display the graph of the deep research engine
        Returns:
            Image: The image of the graph
        """
        image=self.graph.mermaid_image()
        return display(Image(image))

In [134]:
search='# Blind Snakes of the Bahamas\n\nBlind snakes, belonging to the family Typhlopidae, are small, burrowing snakes with a worm-like appearance. Several species are found in the Bahamas, including the Bahamian slender blind snake (Cubatyphlops biminiensis) and the Brown Blind Snake (Typhlops lumbricalis).\n\n## Species\n\n*   **Bahamian Slender Blind Snake (Cubatyphlops biminiensis):** This species is endemic to the Bahamas. Its conservation status is listed as Least Concern.\n*   **Brown Blind Snake (Typhlops lumbricalis):** This species is found on most of the larger islands of the Bahamas, including the Abacos, Andros, the Berry Islands, Eleuthera, Cat Island, Long Island, and throughout the Exuma chain. On Bimini, they are commonly found in rotting logs and under rocks.\n\n## Characteristics\n\nBlind snakes are typically small and slender, resembling worms. They are adapted for burrowing, with smooth scales and reduced eyes.\n\n## Habitat and Distribution\n\nBahamian blind snakes inhabit various environments, including coppice dry forest habitat. They are often found under rocks, in rotting logs, and in the soil.\n\n## Conservation\n\nWhile some blind snake species are listed as Least Concern, they still face threats such as habitat loss and degradation due to human activities like urbanization and agriculture. Conservation efforts are important to ensure the survival of these unique reptiles.\n\n##Interesting facts\n\nIt is really hard to see these SMALL snakes in the Bahamas. Take one look at them, and you will notice they look more like small worms than the other snakes that live in the Bahamas.\n\n## References\n\n*   Animalia.bio: [https://animalia.bio/bahamian-slender-blind-snake](https://animalia.bio/bahamian-slender-blind-snake)\n*   Wikipedia: [https://en.wikipedia.org/wiki/Bahamian_slender_blind_snake](https://en.wikipedia.org/wiki/Bahamian_slender_blind_snake)\n*   Hummingbirdsplus.org: [https://www.hummingbirdsplus.org/nature-blog-network/the-5-types-of-snakes-found-on-the-bahamas/](https://www.hummingbirdsplus.org/nature-blog-network/the-5-types-of-snakes-found-on-the-bahamas/)\n*   Biminisharklab.com: [https://www.biminisharklab.com/fauna-of-bimini-1/bahamian-brown-blind-snake](https://www.biminisharklab.com/fauna-of-bimini-1/bahamian-brown-blind-snake)\n*   journals.ku.edu: [https://journals.ku.edu/reptilesandamphibians/article/download/15667/14618](https://journals.ku.edu/reptilesandamphibians/article/download/15667/14618)\n*   Allenpress.com: [https://meridian.allenpress.com/herpetologica/article/67/2/194/32795/Taxonomy-of-the-Blind-Snakes-Associated-with](https://meridian.allenpress.com/herpetologica/article/67/2/194/32795/Taxonomy-of-the-Blind-Snakes-Associated-with)\n*   Birdwatchinghq.com: [https://birdwatchinghq.com/snakes-of-the-Bahamas/](https://birdwatchinghq.com/snakes-of-the-Bahamas/)\n*   Photoguides.org: [https://photoguides.org/snakes-in-the-bahamas/](https://photoguides.org/snakes-in-the-bahamas/)\n*   Wildexplained.com: [https://wildexplained.com/animal-encyclopedia/the-western-blind-snake-uncovering-the-mystery-of-the-elusive-reptile/](https://wildexplained.com/animal-encyclopedia/the-western-blind-snake-uncovering-the-mystery-of-the-elusive-reptile/)\n*   Wildexplained.com: [https://wildexplained.com/animal-encyclopedia/discovering-the-mysterious-blind-snake/](https://wildexplained.com/animal-encyclopedia/discovering-the-mysterious-blind-snake/)\n\n'

In [135]:
pres=Presentation_gen()

In [136]:
res= await pres.chat(search)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\deep_research_agent\.venv\Lib\site-packages\pydantic\type_adapter.py:527: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `int` with value `1` - serialized value may not be as expected
  return self.serializer.to_python(
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\deep_research_agent\.venv\Lib\site-packages\pydantic\type_adapter.py:527: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `int` with value `1` - serialized value may not be as expected
  Expected `str` but got `int` with value `1` - serialized value may not be as expected
  return self.serializer.to_python(
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\deep_research_agent\.venv\Lib\site-packages\pydantic\type_adapter.py:527: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `int` with value `1` - serialized value may not be as expected
  Expected `str` but got `int` with value `1` - serialized v

In [194]:
pres.state.presentation_plan.tasks

[steps(image_data=image(image_title='Title Slide: Blind Snakes of the Bahamas', image_url=None), text_data=text(Text_to_extract='Introduce the topic of blind snakes in the Bahamas.', Text_title='Introduction', Text_content='Overview of blind snakes (Typhlopidae) and their presence in the Bahamas.')),
 steps(image_data=image(image_title='Species: Bahamian Slender Blind Snake', image_url='https://animalia.bio/bahamian-slender-blind-snake'), text_data=text(Text_to_extract='Detail the species found in the Bahamas, include conservation status', Text_title='Species', Text_content='Bahamian Slender Blind Snake (Cubatyphlops biminiensis): Endemic, Least Concern. [Animalia.bio, Wikipedia]')),
 steps(image_data=image(image_title='Species: Brown Blind Snake', image_url=None), text_data=text(Text_to_extract='Detail the species found in the Bahamas, include the islands where it can be found', Text_title='Species Cont.', Text_content='Brown Blind Snake (Typhlops lumbricalis): Found on Abacos, Andros

In [190]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE


def create_presentation(config: PresentationConfig) -> bool:
    # try:
        # Create presentation
        prs = Presentation()
        
        # Set slide dimensions if different from default
        if config.slide_width != 10.0 or config.slide_height != 7.5:
            prs.slide_width = Inches(10)
            prs.slide_height = Inches(7.5)
        
        # Add title slide
        title_slide_layout = prs.slide_layouts[0]
        slide = prs.slides.add_slide(title_slide_layout)
        title = slide.shapes.title
        subtitle = slide.placeholders[1]
        
        title.text = config.title
        if config.subtitle:
            subtitle.text = config.subtitle
        
        # Process each slide
        for slide_content in config.slides:
            # Determine slide layout based on content
            # if slide_content.chart:
            #     slide_layout = prs.slide_layouts[5]  # Chart layout
            if slide_content.bullet_points:
                slide_layout = prs.slide_layouts[1]  # Bullet points layout
            else:
                slide_layout = prs.slide_layouts[6]  # Blank layout
            
            slide = prs.slides.add_slide(slide_layout)
            
            # Add title if present
            if slide_content.title:
                title = slide.shapes.title
                if title:
                    title.text = slide_content.title
            
            # Add bullet points if present
            if slide_content.bullet_points:
                body_shape = slide.placeholders[1]
                tf = body_shape.text_frame
                
                for bullet in slide_content.bullet_points:
                    p = tf.add_paragraph()
                    p.text = bullet.text
                    p.level = bullet.level
                    
                    # Apply formatting if specified
                    if bullet.format:
                        run = p.runs[0] if p.runs else p.add_run()
                        run.font.size = Pt(bullet.format.font_size)
                        if bullet.format.font_color:
                            run.font.color.rgb = RGBColor(bullet.format.font_color.red, bullet.format.font_color.green, bullet.format.font_color.blue)
                        run.font.bold = bullet.format.is_bold
                        run.font.italic = bullet.format.is_italic
            
            # Add text boxes if present
            if slide_content.text_boxes:
                for text_box in slide_content.text_boxes:
                    left, top, width, height = text_box.position.left, text_box.position.top, text_box.position.width, text_box.position.height
                    shape = slide.shapes.add_textbox(
                        Inches(left), Inches(top),
                        Inches(width), Inches(height)
                    )
                    tf = shape.text_frame
                    tf.auto_size
                    tf.word_wrap=True
                    
                    tf.text = text_box.text
                    
                    
                    # Apply formatting if specified
                    if text_box.format:
                        for paragraph in tf.paragraphs:
                            for run in paragraph.runs:
                                run.font.size = Pt(text_box.format.font_size)
                                if text_box.format.font_color:
                                    run.font.color.rgb = RGBColor(text_box.format.font_color.red, text_box.format.font_color.green, text_box.format.font_color.blue)
                                run.font.bold = text_box.format.is_bold
                                run.font.italic = text_box.format.is_italic
            
            # Add chart if present
            # if slide_content.chart:
            #     chart_data = CategoryChartData()
            #     chart_data.categories = slide_content.chart.categories
                
            #     for series in slide_content.chart.series:
            #         chart_data.add_series(
            #             series['name'],
            #             series['values']
            #         )
                
            #     left, top, width, height = slide_content.chart.position
            #     slide.shapes.add_chart(
            #         getattr(XL_CHART_TYPE, slide_content.chart.type.upper()),
            #         Inches(left), Inches(top),
            #         Inches(width), Inches(height),
            #         chart_data
            #     )
            
            # Add image if present
            if slide_content.image_url and slide_content.image_position:
                left, top, width, height = slide_content.image_position.left, slide_content.image_position.top, slide_content.image_position.width, slide_content.image_position.height
                slide.shapes.add_picture(
                    slide_content.image_url,
                    Inches(left), Inches(top),
                    Inches(width), Inches(height)
                )
        
        # Save the presentation
        prs.save(config.output_path)
        return True
        
    # except Exception as e:
    #     print(f"Error creating presentation: {str(e)}")
    #     return False



In [191]:
if __name__ == "__main__":
    # Example usage
    config = PresentationConfig(
        title="Sample Presentation",
        subtitle="Created with python-pptx",
        output_path="sample.pptx",
        slides=[
            SlideContent(
                title="First Slide",
                bullet_points=[BulletPoint(
                        text="First bullet point",
                        level=0,
                        format=TextFormat(
                            font_size= 12.0,
                            is_bold= True
                            )
                        )   
                ]
            )
        ]
    )
    
    if create_presentation(pres.state.presentation_config):
        print("Presentation created successfully!")
    else:
        print("Error creating presentation")

Presentation created successfully!


In [130]:
config.slides[0].bullet_points[0].format.

TextFormat(font_size=12.0, font_color=Font_color(red=0, green=0, blue=0), is_bold=True, is_italic=False, alignment=1)